In [14]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from skimage.feature import local_binary_pattern, hog
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from xgboost import XGBClassifier

In [2]:
def load_data(dataset_path):
    classes = {'Acne': 0, 'Bags': 1, 'Redness': 2}  # Map class names to labels
    X, y = [], []
    for class_name, label in classes.items():
        class_path = os.path.join(dataset_path, class_name)
        if not os.path.exists(class_path):
            print(f"Folder not found: {class_path}")
            continue
        for subfolder in os.listdir(class_path):  # Iterate through subfolders (0-10)
            subfolder_path = os.path.join(class_path, subfolder)
            if os.path.isdir(subfolder_path):
                for img_name in os.listdir(subfolder_path):  # Iterate through images
                    img_path = os.path.join(subfolder_path, img_name)
                    img = cv2.imread(img_path)
                    if img is not None:
                        img = cv2.resize(img, (128, 128))  # Resize images to 128x128
                        X.append(img)
                        y.append(label)
                    else:
                        print(f"Failed to load image: {img_path}")
    print(f"Loaded {len(X)} images from {len(classes)} classes.")
    return np.array(X), np.array(y)

In [49]:
def extract_lbp_features(img, P=8, R=1):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray, P=P, R=R, method='uniform')
    hist, _ = np.histogram(lbp, bins=np.arange(0, P+3), range=(0, P+2))
    hist = hist.astype('float')
    hist /= (hist.sum() + 1e-6)  # Normalize
    return hist

def extract_color_features(img):
    # Convert to HSV color space
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # Calculate histograms for each channel (H, S, V)
    hist_h = cv2.calcHist([hsv], [0], None, [256], [0, 255])
    hist_s = cv2.calcHist([hsv], [1], None, [256], [100, 175])
    hist_v = cv2.calcHist([hsv], [2], None, [256], [150, 255])
    # Normalize the histograms
    hist_h /= hist_h.sum() + 1e-6
    hist_s /= hist_s.sum() + 1e-6
    hist_v /= hist_v.sum() + 1e-6
    # Flatten and concatenate histograms
    return np.concatenate([hist_h.flatten(), hist_s.flatten(), hist_v.flatten()])

def extract_combined_features(img):
    lbp_features = extract_lbp_features(img)
    color_features = extract_color_features(img)
    return np.concatenate([lbp_features, color_features])

In [ ]:
X, y = load_data('Dataset')

In [47]:
# Extract features and labels
# X_resized = X.reshape(X.shape[0], -1)
X_features = np.array([extract_combined_features(img) for img in X])
# X_features.reshape(X_features.shape[0], -1)

X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2, random_state=42)

In [48]:
# Train an SVM classifier
model = SVC(kernel='linear')
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 45.74%


In [50]:
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import itertools
import cv2
import numpy as np

# Function to extract HOG features
def extract_hog_features(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2)):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features, _ = hog(gray, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, block_norm='L2-Hys', visualize=True)
    return features

# Function to extract color features
def extract_color_features(img, h_range=(0, 256), s_range=(0, 256), v_range=(0, 256)):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hist_h = cv2.calcHist([hsv], [0], None, [256], h_range)
    hist_s = cv2.calcHist([hsv], [1], None, [256], s_range)
    hist_v = cv2.calcHist([hsv], [2], None, [256], v_range)
    hist_h /= hist_h.sum() + 1e-6
    hist_s /= hist_s.sum() + 1e-6
    hist_v /= hist_v.sum() + 1e-6
    return np.concatenate([hist_h.flatten(), hist_s.flatten(), hist_v.flatten()])

# Function to extract combined features
def extract_combined_features(img, h_range=(0, 256), s_range=(0, 256), v_range=(0, 256)):
    hog_features = extract_hog_features(img)
    color_features = extract_color_features(img, h_range, s_range, v_range)
    return np.concatenate([hog_features, color_features])

# Function to find the best HSV ranges
def find_best_hsv_ranges(X, y, ranges):
    best_accuracy = 0
    best_ranges = None
    for h_range, s_range, v_range in itertools.product(ranges, ranges, ranges):
        print(f"Testing ranges H={h_range}, S={s_range}, V={v_range}")
        X_features = np.array([extract_combined_features(img, h_range, s_range, v_range) for img in X])
        X_train, X_val, y_train, y_val = train_test_split(X_features, y, test_size=0.2, random_state=42)
        model = SVC(kernel='linear')
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        accuracy = accuracy_score(y_val, y_pred)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_ranges = (h_range, s_range, v_range)
            print(f"New best accuracy: {accuracy} with ranges H={h_range}, S={s_range}, V={v_range}")
    return best_ranges, best_accuracy

# Example usage
# Assuming X is your dataset of images and y are the corresponding labels
ranges = [(0, 128), (128, 256), (50, 200), (100, 255), (100, 150), (50, 150), (80, 175), (80, 200)]  # Example range candidates
best_ranges, best_accuracy = find_best_hsv_ranges(X, y, ranges)
print("Best Ranges:", best_ranges, "Best Accuracy:", best_accuracy)


Testing ranges H=(0, 128), S=(0, 128), V=(0, 128)
New best accuracy: 0.6595744680851063 with ranges H=(0, 128), S=(0, 128), V=(0, 128)
Testing ranges H=(0, 128), S=(0, 128), V=(128, 256)
Testing ranges H=(0, 128), S=(0, 128), V=(50, 200)
Testing ranges H=(0, 128), S=(0, 128), V=(100, 255)
Testing ranges H=(0, 128), S=(0, 128), V=(100, 150)
Testing ranges H=(0, 128), S=(0, 128), V=(50, 150)
Testing ranges H=(0, 128), S=(0, 128), V=(80, 175)
Testing ranges H=(0, 128), S=(0, 128), V=(80, 200)
Testing ranges H=(0, 128), S=(128, 256), V=(0, 128)
Testing ranges H=(0, 128), S=(128, 256), V=(128, 256)
Testing ranges H=(0, 128), S=(128, 256), V=(50, 200)
Testing ranges H=(0, 128), S=(128, 256), V=(100, 255)
Testing ranges H=(0, 128), S=(128, 256), V=(100, 150)
Testing ranges H=(0, 128), S=(128, 256), V=(50, 150)
Testing ranges H=(0, 128), S=(128, 256), V=(80, 175)
Testing ranges H=(0, 128), S=(128, 256), V=(80, 200)
Testing ranges H=(0, 128), S=(50, 200), V=(0, 128)
Testing ranges H=(0, 128), S